# Broadband Funding Map

This notebook will run through the national Broadband Funding Map APIs. For more background on the national Broadband Funding Map please go *here*

## Getting setup

Install requests if not already available.

In [66]:
!pip install requests pandas itables

                                              0.0/199.3 kB ? eta -:--:--
     ------                                  30.7/199.3 kB 1.4 MB/s eta 0:00:01
     -----------                           61.4/199.3 kB 550.5 kB/s eta 0:00:01
     -----------------                     92.2/199.3 kB 751.6 kB/s eta 0:00:01
     -------------------------            143.4/199.3 kB 711.9 kB/s eta 0:00:01
     -------------------------------      174.1/199.3 kB 700.2 kB/s eta 0:00:01
     ------------------------------------ 199.3/199.3 kB 756.1 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Do the imports

In [71]:
import requests, json, itables
from pprint import pprint
import pandas as pd

itables.init_notebook_mode(all_interactive=False)
itables.options.maxBytes = 0

<IPython.core.display.Javascript object>

## Get the current UUID

We may still be subject to rate limiting. Lets check!

In [40]:
url = "https://fundingmap.fcc.gov/bfm/api/map/getPublishedRun"
r = requests.get(url, timeout=5)

ReadTimeout: HTTPSConnectionPool(host='fundingmap.fcc.gov', port=443): Read timed out. (read timeout=5)

Yep... lets implement a session and set our header, and try to get the published run.

In [41]:
s = requests.Session()
s.headers.update({'User-Agent': 'bfm-explorer 0.0.1'})

r = s.get(url)

Cool, it worked lets move on.

Lets get the published run information.

In [42]:
print(url)

r = s.get(url)

https://fundingmap.fcc.gov/bfm/api/map/getPublishedRun


And parse it pulling out the uuid.

In [43]:
parsed = json.loads(r.content)

pprint(parsed)

uuid = parsed['data'][0]['process_uuid']

print(f'\nUUID: {uuid}')

{'data': [{'create_date': '2023-08-02',
           'home_page_config': 'br=r&speed=25_3&tech=1_2_3_6_7',
           'nbm_create_date': '2023-07-25',
           'nbm_data_as_of_date': '2022-12-31',
           'nbm_process_uuid': '2750f3d2-3b48-4a96-9e52-2a21eb79d5ed',
           'process_uuid': '50b888a0-df10-40bb-ac6e-893caa3a8b3c'}],
 'message': None,
 'request_date': '2023-08-03T16:45:52.461Z',
 'result_count': 1,
 'status': 'successful',
 'status_code': 200}

UUID: 50b888a0-df10-40bb-ac6e-893caa3a8b3c


## Funding Agencies

This is a pretty easy and straight forward one.

In [48]:
url = f"https://fundingmap.fcc.gov/bfm/api/map/getAgencyList/{uuid}"

r = s.get(url)

parsed = json.loads(r.content)

parsed

{'data': [{'id': '1', 'agency_name': 'Federal Communications Commission'},
  {'id': '5', 'agency_name': 'NTIA'},
  {'id': '3', 'agency_name': 'Rural Utilities Service'},
  {'id': '4', 'agency_name': 'US Department of Treasury'}],
 'result_count': 4,
 'status_code': 200,
 'message': None,
 'status': 'successful',
 'request_date': '2023-08-03T17:36:46.076Z'}

Lets get a list of Agencies.

In [45]:
agencies = [item['agency_name'] for item in parsed['data']]

agencies

['Federal Communications Commission',
 'NTIA',
 'Rural Utilities Service',
 'US Department of Treasury']

## Programs

Lets find out what programs are available.

In [51]:
url = f"https://fundingmap.fcc.gov/bfm/api/map/getDataDownload/{uuid}"

r = s.get(url)

parsed = json.loads(r.content)

parsed

{'data': [{'id': 11036,
   'data_category': 'Funding Data',
   'data_type': 'Program',
   'agency_id': 1,
   'agency_name': 'Federal Communications Commission',
   'program_id': 25,
   'program_name': 'Bringing Puerto Rico Together',
   'project_id': None,
   'project_name': None,
   'state_fips': None,
   'state_name': None,
   'file_type': 'csv',
   'modified_date': None,
   'file_name': 'fundingdata_program25_20230803',
   'download_available': 'Yes'},
  {'id': 11041,
   'data_category': 'Funding Data',
   'data_type': 'Program',
   'agency_id': 1,
   'agency_name': 'Federal Communications Commission',
   'program_id': 28,
   'program_name': 'Connect America Fund Phase II',
   'project_id': None,
   'project_name': None,
   'state_fips': None,
   'state_name': None,
   'file_type': 'csv',
   'modified_date': None,
   'file_name': 'fundingdata_program28_20230803',
   'download_available': 'Yes'},
  {'id': 11037,
   'data_category': 'Funding Data',
   'data_type': 'Program',
   'agenc

In [52]:
programs = {item['program_name'] for item in parsed['data']}

In [53]:
programs

{'Bringing Puerto Rico Together',
 'Broadband Infrastructure Program',
 'COMMUNITY CONNECT GRANT PROGRAM',
 'Capital Projects Fund',
 'Connect America Fund Phase II',
 'Connect USVI',
 None,
 'RURAL ECONNECTIVITY PROGRAM',
 'Rural Digital Opportunity Fund',
 'TELEPHONE LOAN PROGRAM',
 'Tribal Broadband Connectivity Program NOFO 1'}

Let's jump into pandas and dig a little deeper.

In [56]:
df = pd.DataFrame(parsed['data'])

Here is the whole table. Feel free to search it for specific programs.

In [73]:
itables.show(df)

Quick EDA

In [76]:
df.data_category.value_counts()

df[df.data_category == 'Unserved-Unfunded']

,id,data_category,data_type,agency_id,agency_name,program_id,program_name,project_id,project_name,state_fips,state_name,file_type,modified_date,file_name,download_available
1063,10979,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,01,Alabama,csv,None,state01_unserved_unfunded_20230803,Yes
1064,10980,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,02,Alaska,csv,None,state02_unserved_unfunded_20230803,Yes
1065,10981,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,04,Arizona,csv,None,state04_unserved_unfunded_20230803,Yes
1066,10982,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,05,Arkansas,csv,None,state05_unserved_unfunded_20230803,Yes
1067,10983,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,06,California,csv,None,state06_unserved_unfunded_20230803,Yes
1068,10984,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,08,Colorado,csv,None,state08_unserved_unfunded_20230803,Yes
1069,10985,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,09,Connecticut,csv,None,state09_unserved_unfunded_20230803,Yes
1070,10986,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,10,Delaware,csv,None,state10_unserved_unfunded_20230803,Yes
1071,10987,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,11,District of Columbia,csv,None,state11_unserved_unfunded_20230803,Yes
1072,10988,Unserved-Unfunded,State,NaN,None,NaN,None,NaN,None,12,Florida,csv,None,state12_unserved_unfunded_20230803,Yes


Pulling out the Unserved-Unfunded data shows us these are the state level files. These are pretty straightforward.